In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import json

from torch.utils.data import Dataset
import pandas as pd
import os
import re

In [0]:
torch.__version__

'1.4.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
 alpha-dataset
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
 Arslan_label_falling
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
 colored_dataset_Arslan
 coloured_dataset_Govi
 extract_faceandhand_jorehgaye.ipynb
 Found_missing_detections
 GrayScale_dataset_Arslan
 GrayScale_dataset_Govi
 loss
 MissingDetections_Arslan_colored
 MissingDetections_Govi_coloured
 putTXT_rightplace.ipynb
 TestingModels_first.ipynb
 TestingModels_second.ipynb
 weights


In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        #transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        #transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

In [0]:
class FallingDataset(Dataset):
    """Falling dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        print("csv ",csv_file)
        self.dataframe = pd.read_csv(csv_file)
        print(self.dataframe)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def natural_sort(self,l): 
        convert = lambda text: int(text) if text.isdigit() else text.lower() 
        alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
        return sorted(l, key = alphanum_key)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Path to image
        video_path = os.path.join(self.root_dir,self.dataframe.iloc[idx, 0])
        
        # Flip flag
        flip = False
        if( self.dataframe.iloc[idx,1] == '1'):   
            flip = True
        
        label = int(self.dataframe.iloc[idx,2])

        frames = []
        heatmaps_face = []
        heatmaps_hand = []
        heatmaps_pose = []
        all_frame_names = os.listdir(video_path)
        all_frame_names = self.natural_sort(all_frame_names)
        count = 0
        for frame_path in all_frame_names:
            
            if '.jpg' in frame_path:
                count += 1
                image_path = self.root_dir + self.dataframe.iloc[idx, 0] + '/' + frame_path
                frame = Image.open( image_path ).convert('L')
                
                if flip == True:  # horizontal flip 
                    frame = frame.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                
                # Get annotations heatmaps
                if label == 0:
                    lbl_folder = '/Falling_Label/'
                    json_folder = '/Falling_Jsons/'
                else:
                    lbl_folder = '/NonFalling_Label/'
                    json_folder= '/NonFalling_Jsons/'

                ann_path = './alpha-datset' + lbl_folder + frame_path.split('.')[0] + '.txt'
                img_w , img_h = frame.size
                hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)

                ann_path = './alpha-datset' + json_folder + frame_path.split('.')[0] + '.json'
                hm_pose = self.get_heatmap_pose(ann_path ,img_w,img_h)

                frame = frame.resize((224,224)) # Resize
                hm_hand = hm_hand.resize((224,224))
                hm_face = hm_face.resize((224,224))
                #hm_pose = hm_pose.resize((224,224))
                  
                frames.append( np.array(frame) )
                heatmaps_hand.append( np.array(hm_hand) )
                heatmaps_face.append( np.array(hm_face) )
                
                for i in range(len(hm_pose)):
                    hm_pose[i] = np.array(hm_pose[i].resize((224,224)))
                
                #print( (np.stack(hm_pose,axis=2)).shape )
                heatmaps_pose.append( np.stack(hm_pose,axis=2) )
            if count == 10:
                break            
        # Stack frames
        # Stack frames
        frames_stack = np.stack(frames, axis=2)
        heatmaps_hand_stack = np.stack(heatmaps_hand, axis=2)
        heatmaps_face_stack = np.stack(heatmaps_face, axis=2)
        
        

        heatmaps_pose_stack = np.concatenate(heatmaps_pose, axis =2)
        #np.stack(heatmaps_pose, axis=2)
        #print(heatmaps_pose_stack.shape)
        # print("frames_s: ",frames_stack.shape)
        # print("hand_maps_s: ",heatmaps_hand_stack.shape)
        # print("face_maps_s: ",heatmaps_face_stack.shape)

        image = np.concatenate((frames_stack,heatmaps_face_stack,heatmaps_hand_stack,heatmaps_pose_stack),axis=2)

      

#        image = np.stack((frames,heatmaps_face,heatmaps_hand,heatmaps_pose), axis=2)
        #print(image.shape)
        
        #sample = {'image': image, 'label': int(self.dataframe.iloc[idx,2])}

        
        
        if self.transform:
            image = self.transform(image)
                      
        return image, label


    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

    def get_heatmap_pose(self,ann_path, img_w, img_h):
        
        heatmaps = [ Image.new('L', (img_w, img_h)) for x in range(17) ]

        try:
            with open(ann_path,'r') as f:
                data = json.load(f)
            #print(ann_path)
            #print(data)  
            try:
                keypoints = data[0]['keypoints']
            except IndexError:
                return heatmaps

            #print(keypoints)
            # TODO check for two key mappings -- working

            for i in range(0,len(keypoints),3):

                xmin = keypoints[i] - 5
                ymin = keypoints[i+1] - 5
                xmax = keypoints[i] + 5
                ymax = keypoints[i+1] + 5

                hmDraw = ImageDraw.Draw(heatmaps[i//3]) 
                hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")

        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return heatmaps


In [6]:
input_path = './alpha-dataset'

image_datasets = {
    'train': 
    FallingDataset(input_path + '/train.csv', input_path + '/color_govi_train/',  data_transforms['train']),
    'validation': 
    FallingDataset(input_path + '/val.csv', input_path + '/color_govi_val/' , data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=8,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=16,
                                shuffle=True,
                                num_workers=4)  # for Kaggle
}

csv  ./alpha-dataset/train.csv
                                  path  flip  label
0    NonFalling/VID_20191201_225548757     0      1
1    NonFalling/VID_20191201_225755268     0      1
2    NonFalling/VID_20191201_225645213     0      1
3    NonFalling/VID_20191201_225510266     0      1
4    NonFalling/VID_20191201_225340330     0      1
..                                 ...   ...    ...
129                   Falling/IMG_5759     0      0
130                   Falling/IMG_5765     0      0
131                   Falling/IMG_5761     0      0
132                   Falling/IMG_5769     0      0
133                   Falling/IMG_5767     0      0

[134 rows x 3 columns]
csv  ./alpha-dataset/val.csv
                                 path  flip  label
0   NonFalling/VID_20191201_225932433     0      1
1   NonFalling/VID_20191201_225958706     0      1
2   NonFalling/VID_20191201_225855345     0      1
3   NonFalling/VID_20191201_225833039     0      1
4   NonFalling/VID_20191201_225814681

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = models.resnet50(pretrained=True).to(device)
model.cuda()
#print(model)

depth = 30 + (17*10)    


for param in model.parameters():
    param.requires_grad = False   


#------------ Modify First Layer
model.conv1 = nn.Conv2d(depth, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
model.conv1.requires_grad = True

#------------ Modify Last Layer
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        
        f = open('loss/beta/raw-face-hand-pose.txt','a+')
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        f.write('Epoch {}/{}\n'.format(epoch+1, num_epochs))
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

                  _, preds = torch.max(outputs, 1)
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

                  count = count+1                
                  if (count%40==0):
                    print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            f.write('{} loss: {:.4f}, acc: {:.4f}\n'.format(phase,epoch_loss,epoch_acc))
            
        torch.save(model, 'weights/beta/raw-face-hand-pose/epoch_%d.pth'%(epoch))
        f.close()


    return model

In [0]:
!pwd


model_trained = train_model(model, criterion, optimizer, num_epochs=100)


/content/drive/My Drive/Resnet_Beta-Version
Epoch 1/100
----------
train loss: 0.9133, acc: 0.4552
validation loss: 0.6965, acc: 0.4630
Epoch 2/100
----------
train loss: 0.6813, acc: 0.5672
validation loss: 0.7022, acc: 0.5000
Epoch 3/100
----------
train loss: 0.6380, acc: 0.6269
validation loss: 0.6907, acc: 0.5185
Epoch 4/100
----------
train loss: 0.6081, acc: 0.6269
validation loss: 0.6923, acc: 0.5370
Epoch 5/100
----------
train loss: 0.5714, acc: 0.7463
validation loss: 0.6496, acc: 0.6296
Epoch 6/100
----------
train loss: 0.5413, acc: 0.7463
validation loss: 0.6494, acc: 0.6296
Epoch 7/100
----------
train loss: 0.5017, acc: 0.7388
validation loss: 0.6800, acc: 0.5926
Epoch 8/100
----------
train loss: 0.5048, acc: 0.7313
validation loss: 1.0418, acc: 0.5370
Epoch 9/100
----------
train loss: 0.5819, acc: 0.6866
validation loss: 0.6548, acc: 0.6481
Epoch 10/100
----------
train loss: 0.4581, acc: 0.7836
validation loss: 0.6924, acc: 0.6111
Epoch 11/100
----------
train loss:

In [0]:
torch.save(model_ trained.state_dict(), 'weights/pytorch/weightsE3.h5')

In [0]:
# model = models.resnet50(pretrained=False).to(device)
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)
epoch = 99
model = (torch.load('weights/raw-face-hand-pose-d/rfhp_epoch_%d.pth'%epoch))

In [0]:
!ls

darknet		      examples	      opt.py		train_sppe
data		      fallmodel.pth   output		video_ann.py
datalabeler_101.py    fi_data	      PoseFlow		video_demo.py
datalabeler_111.py    fn.py	      pPose_nms.py	webcam_demo.py
datalabeler_249.py    img.png	      __pycache__	weights
dataloader.py	      LICENSE	      README.md		yolo
dataloader_webcam.py  matching.py     requirements.txt
demo.py		      models	      resnetWeights
doc		      online_demo.py  SPPE


In [0]:
import glob
import random
pathFalling = '/content/drive/My Drive/Resnet_Beta-Version/alpha-dataset/color_govi_test/Falling/IMG_5785'
pathNonFalling = '/content/drive/My Drive/Resnet_Beta-Version/alpha-dataset/color_govi_test/NonFalling/IMG_5785'
accurateFalling = 0
accurateNonFalling = 0

files= os.listdir(pathFalling)

list1 = []
for f in files:
  if "jpg" in f:
    list1.append(os.path.join(pathFalling,f))

print (list1)

files= os.listdir(pathNonFalling)

list2 = []
for f in files:
  if "jpg" in f:
    list2.append(os.path.join(pathNonFalling,f))

print (list2)


#list1= [f for f in glob.glob(pathFalling + "*.jpg", recursive=True)]
#list2= [f for f in glob.glob(pathNonFalling + "*.jpg", recursive=True)]
validation_img_pathsFalling=random.sample(list1,10)
validation_img_pathsNonFalling = random.sample(list2,10)
img_listFalling = [Image.open('' + img_path) for img_path in validation_img_pathsFalling]
img_listNonFalling = [Image.open('' + img_path) for img_path in validation_img_pathsNonFalling]
print (len(img_listFalling))
print (len(img_listNonFalling))

NameError: ignored

In [0]:
validation_batchFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listFalling])
validation_batchNonFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listNonFalling])

In [0]:
pred_logits_tensorFalling = model(validation_batchFalling)
pred_logits_tensorFalling
pred_logits_tensorNonFalling = model(validation_batchNonFalling)
pred_logits_tensorNonFalling

RuntimeError: ignored

In [0]:
pred_probsFalling = F.softmax(pred_logits_tensorFalling, dim=1).cpu().data.numpy()
print (pred_probsFalling)
pred_probsNonFalling = F.softmax(pred_logits_tensorNonFalling, dim=1).cpu().data.numpy()
print (pred_probsNonFalling)

[[0.44844374 0.5515562 ]
 [0.12725133 0.8727487 ]
 [0.09804448 0.9019555 ]
 [0.00425655 0.99574345]
 [0.8194356  0.18056442]
 [0.24643923 0.7535607 ]
 [0.13835269 0.8616473 ]
 [0.1574074  0.84259254]
 [0.8485861  0.1514139 ]
 [0.32346088 0.6765391 ]
 [0.41357878 0.58642125]
 [0.15117285 0.8488271 ]
 [0.6003375  0.39966255]
 [0.02290936 0.9770906 ]
 [0.10023551 0.89976454]
 [0.29872367 0.70127636]
 [0.04371396 0.956286  ]
 [0.61934406 0.38065597]
 [0.30597326 0.6940267 ]
 [0.16753031 0.8324697 ]
 [0.66433    0.33567002]
 [0.84191    0.15808995]
 [0.6628018  0.33719823]
 [0.4977156  0.50228435]
 [0.87176895 0.12823105]
 [0.02149463 0.9785054 ]
 [0.48655316 0.5134468 ]
 [0.44041517 0.55958486]
 [0.1114507  0.88854927]
 [0.5616096  0.4383904 ]
 [0.570884   0.42911607]
 [0.32838956 0.6716105 ]
 [0.42478982 0.5752102 ]
 [0.5371839  0.46281612]
 [0.02694354 0.9730565 ]
 [0.61242115 0.3875789 ]
 [0.21000993 0.7899901 ]
 [0.32981074 0.6701893 ]
 [0.23768397 0.76231605]
 [0.846486   0.15351407]


In [0]:
i=0
accurateFalling=0
for i in range(len(pred_probsFalling)):
  if (pred_probsFalling[i][0]> 0.5):
    accurateFalling= accurateFalling +1 
i=0
accurateNonFalling=0
for i in range(len(pred_probsNonFalling)):
  if (pred_probsNonFalling[i][1]> 0.5):
    accurateNonFalling= accurateNonFalling +1 


In [0]:
overallAccuracy = (accurateNonFalling+accurateFalling)/(len(img_listFalling)+len(img_listNonFalling))

In [0]:
print (accurateFalling)
print (accurateNonFalling)

41
92


In [0]:
print (overallAccuracy)

0.665


In [0]:
ig, axs = plt.subplots(1, len(img_listFalling), figsize=(150, 150))
for i, img in enumerate(img_listFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsFalling[i,0],
                                                          100*pred_probsFalling[i,1]))
    ax.imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
ig, axs = plt.subplots(1, len(img_listNonFalling), figsize=(150, 150))
for i, img in enumerate(img_listNonFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsNonFalling[i,0],
                                                          100*pred_probsNonFalling[i,1]))
    ax.imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
epoch = 99
device = 'cuda'
model = torch.load('weights/raw-face-hand-pose-d/rfhp_epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()

ResNet(
  (conv1): Conv2d(22, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [0]:
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 60
total 100
0.6
